In [8]:
import pandas as pd
from datetime import datetime

def generate_vendor_scorecard():
    # === STEP 1: Load Data ===
    try:
        df = pd.read_csv("../data/cleaned_message.csv")  
        ner_df = pd.read_csv("../data/tokenized_messages.csv")
    except FileNotFoundError as e:
        print(f"Error loading data files: {e}")
        return None

    # === STEP 2: Merge Data ===
    # Ensure we have required columns
    required_columns = {'ID', 'Date'}
    if not required_columns.issubset(df.columns) or not required_columns.issubset(ner_df.columns):
        print("Missing required columns in input data")
        return None

    merged = pd.merge(df, ner_df, on=['ID'], how='left', suffixes=('', '_ner'))
    
    # Convert and clean date
    merged['Date'] = pd.to_datetime(merged['Date'], errors='coerce')
    merged = merged.dropna(subset=['Date'])
    
    if merged.empty:
        print("No valid data after merging and cleaning")
        return None

    # === STEP 3: Compute Metrics Per Vendor ===
    vendor_scores = []

    for vendor, group in merged.groupby('ID'):
        group = group.sort_values('Date')
        
        # Activity & Consistency
        time_span = (group['Date'].max() - group['Date'].min()).days
        days_range = time_span if time_span > 0 else 1  # avoid division by zero
        posts_per_week = round((len(group) / days_range) * 7, 2)

        # Engagement (handle missing views)
        avg_views = group['views'].mean() if 'views' in group and not group['views'].isnull().all() else 0
        
        # Top post info
        top_post = group.loc[group['views'].idxmax()] if not group['views'].isnull().all() else {}
        top_product = top_post.get('product', 'N/A')
        top_price = top_post.get('price', 'N/A')

        # Business Profile (price analysis)
        prices = pd.to_numeric(group['price'], errors='coerce')
        valid_prices = prices.dropna()
        avg_price = valid_prices.mean() if not valid_prices.empty else 0

        # Normalized scores (0-100 range)
        views_score = min(avg_views / 1000 * 100, 100)  # assuming 1000 views = max score
        activity_score = min(posts_per_week * 10, 100)   # assuming 10 posts/week = max score
        
        # Weighted lending score
        lending_score = (views_score * 0.6) + (activity_score * 0.4)

        vendor_scores.append({
            "Vendor ID": vendor,
            "Avg. Views/Post": round(avg_views, 2),
            "Posts/Week": posts_per_week,
            "Avg. Price (ETB)": round(avg_price, 2) if avg_price else "N/A",
            "Top Product": top_product,
            "Top Price": top_price,
            "Lending Score": round(lending_score, 1)  # one decimal place
        })

    # === STEP 4: Output Scorecard ===
    scorecard_df = pd.DataFrame(vendor_scores)
    scorecard_df = scorecard_df.sort_values("Lending Score", ascending=False)
    
    try:
        scorecard_df.to_csv("../data/vendor_scorecard.csv", index=False)
        print("Successfully generated vendor scorecard")
        return scorecard_df
    except Exception as e:
        print(f"Error saving scorecard: {e}")
        return None

# Execute the function
if __name__ == "__main__":
    result = generate_vendor_scorecard()
    if result is not None:
        print(result.head())

KeyError: 'views'

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

# Constants (adjust weights based on business priorities)
VIEWS_WEIGHT = 0.5      # Engagement importance
FREQUENCY_WEIGHT = 0.3  # Activity importance
PRICE_WEIGHT = 0.2      # Business profile importance

def calculate_lending_score(avg_views, posts_per_week, avg_price, max_views=5000, max_freq=20, max_price=10000):
    """Normalize metrics to 0-100 scale and compute weighted score."""
    views_norm = min(avg_views / max_views * 100, 100)
    freq_norm = min(posts_per_week / max_freq * 100, 100)
    price_norm = min(avg_price / max_price * 100, 100) if avg_price > 0 else 0
    
    return (views_norm * VIEWS_WEIGHT) + (freq_norm * FREQUENCY_WEIGHT) + (price_norm * PRICE_WEIGHT)

def generate_scorecard():
    # Load data
    try:
        posts_df = pd.read_csv("../data/cleaned_message.csv")  # Columns: ID, Date, message, views
        ner_df = pd.read_csv("../data/tokenized_messages.csv")  # Columns: ID, product, price
    except Exception as e:
        print(f"Data loading failed: {e}")
        return None

    # Merge and clean data
    merged = pd.merge(posts_df, ner_df, on="ID", how="left")
    merged["Date"] = pd.to_datetime(merged["Date"], errors="coerce")
    merged = merged.dropna(subset=["Date", "views"])
    
    # Calculate metrics per vendor
    metrics = []
    for vendor_id, group in merged.groupby("ID"):
        group = group.sort_values("Date")
        
        # Activity & Consistency
        days_active = (group["Date"].max() - group["Date"].min()).days
        posts_per_week = (len(group) / max(days_active, 1)) * 7  # Prevent division by zero
        
        # Engagement
        avg_views = group["views"].mean()
        top_post = group.loc[group["views"].idxmax()]
        
        # Business Profile
        prices = pd.to_numeric(group["price"], errors="coerce").dropna()
        avg_price = prices.mean() if not prices.empty else 0
        
        # Calculate Lending Score
        score = calculate_lending_score(avg_views, posts_per_week, avg_price)
        
        metrics.append({
            "Vendor ID": vendor_id,
            "Avg. Views/Post": round(avg_views),
            "Posts/Week": round(posts_per_week, 1),
            "Avg. Price (ETB)": round(avg_price) if avg_price > 0 else "N/A",
            "Top Product": top_post.get("product", "N/A"),
            "Top Product Views": top_post["views"],
            "Lending Score": round(score, 1)  # 0-100 scale
        })

    # Generate and save scorecard
    scorecard = pd.DataFrame(metrics).sort_values("Lending Score", ascending=False)
    scorecard.to_csv("../data/vendor_scorecard.csv", index=False)
    return scorecard

# Execute
if __name__ == "__main__":
    scorecard = generate_scorecard()
    print("\n=== Top 5 Vendors ===")
    print(scorecard.head().to_string(index=False))

KeyError: 'Date'